# 1. 데이터 전처리

## 1. 연결,개별 합치기

In [2]:
import pandas as pd

In [15]:
# 1) ts2000 csv 가져오기
KOSDAQ_data= pd.read_csv('./data/KOSDAQ_data.csv',encoding='CP949')
KOSDAQ_data_개별 = pd.read_csv('./data/(개별)KOSDAQ_data.csv',encoding='CP949')

In [16]:
# 2) 연결 재무제표 기준 개별 합치기
col = pd.Series(KOSDAQ_data.columns)

# 연결에 없는 값 개별에 있으면 대입
for i in range(len(col)):
        KOSDAQ_data.loc[KOSDAQ_data[col[i]].isna(), col[i]] = KOSDAQ_data_개별[col[i]]

In [17]:
# 3) 거래소 코드 zfill
KOSDAQ_data['거래소코드'] = KOSDAQ_data['거래소코드'].astype('str')
KOSDAQ_data['거래소코드'] = KOSDAQ_data['거래소코드'].str.zfill(6)

In [18]:
# 4) 거래소코드가 2개 이상인거 확인
daq = KOSDAQ_data.groupby(['회사명'])['거래소코드'].nunique().reset_index()
daq[daq['거래소코드']>1]

,회사명,거래소코드
1205,(주)핸디소프트,2


In [19]:
# 4) 삭제
KOSDAQ_data.drop(KOSDAQ_data[KOSDAQ_data['거래소코드']=='032380'].index,inplace=True)
KOSDAQ_data.reset_index(drop=True,inplace=True)

In [20]:
# 5) 회계년도 정리
KOSDAQ_data['회계년도'] = pd.to_datetime(KOSDAQ_data['회계년도'],format='%Y/%m')
KOSDAQ_data['회계년도'] = KOSDAQ_data['회계년도'].dt.year

In [8]:
# 파생변수 전 csv
# KOSDAQ_data.to_csv('./data/kosdaq.csv')

In [21]:
# 6) 4개 미만 즉 회사명, 거래소코드, 회계년도만 존재하는 행 삭제
KOSDAQ_data = KOSDAQ_data.dropna(axis=0,thresh=4)

In [22]:
KOSDAQ_data.isna().sum()

회사명             0
거래소코드           0
회계년도            0
자산              0
유동자산            0
재고자산            0
매출채권및기타유동채권     0
비유동자산           0
유형자산            0
부채              0
유동부채            0
장기차입금           0
단기차입금           0
장기사채            0
단기사채            0
유동성장기부채         0
자본              0
매출원가            0
매출액             0
당기순이익           0
판매비와관리비         0
(정상)영업손익        0
법인세비용차감전순이익     0
급여              0
세금과공과           0
이자비용            0
기말자본금           0
기초자본금           0
유무형자산등상각비       0
대손상각비           0
감가상각비           0
영업활동으로인한현금흐름    0
dtype: int64

In [24]:
KOSDAQ_data

,회사명,거래소코드,회계년도,자산,유동자산,재고자산,매출채권및기타유동채권,비유동자산,유형자산,부채,...,법인세비용차감전순이익,급여,세금과공과,이자비용,기말자본금,기초자본금,유무형자산등상각비,대손상각비,감가상각비,영업활동으로인한현금흐름
0,(주)CMG제약,058820,2010,23456619.0,9965208.0,1946644.0,5063612.0,13491411.0,9055560.0,15723286.0,...,-4187367.0,1791309.0,89350.0,972784.0,10914543.0,8292165.0,1321.0,11516.0,1065106.0,-1388441.0
1,(주)CMG제약,058820,2011,29477750.0,14760027.0,3352328.0,8131871.0,14717723.0,9910510.0,9461282.0,...,572373.0,1879490.0,89667.0,0.0,21846600.0,20588188.0,1020.0,0.0,678655.0,-676550.0
2,(주)CMG제약,058820,2012,45515723.0,33764182.0,2420524.0,10259505.0,11751541.0,9612748.0,29843038.0,...,-3363708.0,0.0,0.0,507788.0,22678546.0,21846600.0,919.0,379207.0,594065.0,-1768573.0
3,(주)CMG제약,058820,2013,56997644.0,40742605.0,4848894.0,11327559.0,16255039.0,10688791.0,7997418.0,...,571839.0,2040204.0,113308.0,1103599.0,43257754.0,33012770.0,832.0,235350.0,87984.0,-3640506.0
4,(주)CMG제약,058820,2014,57646794.0,28706417.0,4802462.0,13343937.0,28940377.0,18912799.0,8398498.0,...,266439.0,2309659.0,158383.0,89925.0,43313433.0,43313433.0,1003.0,172125.0,113135.0,93111.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14025,흥구석유(주),024060,2017,79097665.0,14997406.0,2433279.0,5249730.0,64100260.0,37881308.0,10285654.0,...,1348425.0,0.0,0.0,0.0,1500000.0,1500000.0,0.0,0.0,0.0,1163482.0
14026,흥구석유(주),024060,2018,77001426.0,12707368.0,3107658.0,8168930.0,64294058.0,37836314.0,7865391.0,...,1242949.0,0.0,0.0,83749.0,1500000.0,1500000.0,0.0,0.0,0.0,-289684.0
14027,흥구석유(주),024060,2019,82746159.0,21832487.0,2186636.0,3812630.0,60913672.0,34913731.0,6572966.0,...,9872838.0,0.0,0.0,0.0,1500000.0,1500000.0,0.0,0.0,0.0,1238029.0
14028,흥구석유(주),024060,2020,81242681.0,25107354.0,1869624.0,2579206.0,56135327.0,30211463.0,5596818.0,...,1053674.0,0.0,0.0,0.0,1500000.0,1500000.0,0.0,0.0,0.0,3685255.0


## 2-1. 산업분류 합치기

In [23]:
# 1) 재무비율변수에 산업분류프레임 합치기
industry = pd.read_csv('./data/산업분류_final.csv',encoding='CP949')
industry['회계년도'] = pd.to_datetime(industry['회계년도'],format='%Y')
industry['회계년도'] = industry['회계년도'].dt.year
industry['거래소코드'] = industry['거래소코드'].astype('str').str.zfill(6)
industry = industry[['거래소코드','회계년도','산업분류']]
df_industry = pd.merge(KOSDAQ_data,industry, on = ['거래소코드','회계년도'], how = 'left')
df_industry

,회사명,거래소코드,회계년도,자산,유동자산,재고자산,매출채권및기타유동채권,비유동자산,유형자산,부채,...,급여,세금과공과,이자비용,기말자본금,기초자본금,유무형자산등상각비,대손상각비,감가상각비,영업활동으로인한현금흐름,산업분류
0,(주)CMG제약,058820,2010,23456619.0,9965208.0,1946644.0,5063612.0,13491411.0,9055560.0,15723286.0,...,1791309.0,89350.0,972784.0,10914543.0,8292165.0,1321.0,11516.0,1065106.0,-1388441.0,NaN
1,(주)CMG제약,058820,2011,29477750.0,14760027.0,3352328.0,8131871.0,14717723.0,9910510.0,9461282.0,...,1879490.0,89667.0,0.0,21846600.0,20588188.0,1020.0,0.0,678655.0,-676550.0,제조업
2,(주)CMG제약,058820,2012,45515723.0,33764182.0,2420524.0,10259505.0,11751541.0,9612748.0,29843038.0,...,0.0,0.0,507788.0,22678546.0,21846600.0,919.0,379207.0,594065.0,-1768573.0,제조업
3,(주)CMG제약,058820,2013,56997644.0,40742605.0,4848894.0,11327559.0,16255039.0,10688791.0,7997418.0,...,2040204.0,113308.0,1103599.0,43257754.0,33012770.0,832.0,235350.0,87984.0,-3640506.0,제조업
4,(주)CMG제약,058820,2014,57646794.0,28706417.0,4802462.0,13343937.0,28940377.0,18912799.0,8398498.0,...,2309659.0,158383.0,89925.0,43313433.0,43313433.0,1003.0,172125.0,113135.0,93111.0,제조업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13744,흥구석유(주),024060,2017,79097665.0,14997406.0,2433279.0,5249730.0,64100260.0,37881308.0,10285654.0,...,0.0,0.0,0.0,1500000.0,1500000.0,0.0,0.0,0.0,1163482.0,도매 및 소매업
13745,흥구석유(주),024060,2018,77001426.0,12707368.0,3107658.0,8168930.0,64294058.0,37836314.0,7865391.0,...,0.0,0.0,83749.0,1500000.0,1500000.0,0.0,0.0,0.0,-289684.0,도매 및 소매업
13746,흥구석유(주),024060,2019,82746159.0,21832487.0,2186636.0,3812630.0,60913672.0,34913731.0,6572966.0,...,0.0,0.0,0.0,1500000.0,1500000.0,0.0,0.0,0.0,1238029.0,도매 및 소매업
13747,흥구석유(주),024060,2020,81242681.0,25107354.0,1869624.0,2579206.0,56135327.0,30211463.0,5596818.0,...,0.0,0.0,0.0,1500000.0,1500000.0,0.0,0.0,0.0,3685255.0,도매 및 소매업


In [626]:
# 임시저장
# df_industry.to_csv('./csv/재무산업.csv')

## 2-2. 감사의견 더하기

In [25]:
# 1) 데이터 처리
op = pd.read_csv('./data/감사의견.csv',encoding='CP949', index_col = 0)
op['거래소코드'] = op['거래소코드'].astype('str').str.zfill(6)
op['회계년도'] = pd.to_datetime(op['회계년도'],format='%Y')
op['회계년도'] = op['회계년도'].dt.year
op.rename(columns = {'감사의견코드(IFRS)':'감사의견코드'}, inplace = True)
op= op[['거래소코드','회계년도','감사의견코드']]
# 2) 산업까지 플러스 된거에 opinion 합치기
df_industry_op = pd.merge(df_industry, op, on = ['거래소코드','회계년도'], how = 'left')
df_industry_op

,회사명,거래소코드,회계년도,자산,유동자산,재고자산,매출채권및기타유동채권,비유동자산,유형자산,부채,...,세금과공과,이자비용,기말자본금,기초자본금,유무형자산등상각비,대손상각비,감가상각비,영업활동으로인한현금흐름,산업분류,감사의견코드
0,(주)CMG제약,058820,2010,23456619.0,9965208.0,1946644.0,5063612.0,13491411.0,9055560.0,15723286.0,...,89350.0,972784.0,10914543.0,8292165.0,1321.0,11516.0,1065106.0,-1388441.0,NaN,NaN
1,(주)CMG제약,058820,2011,29477750.0,14760027.0,3352328.0,8131871.0,14717723.0,9910510.0,9461282.0,...,89667.0,0.0,21846600.0,20588188.0,1020.0,0.0,678655.0,-676550.0,제조업,UQ
2,(주)CMG제약,058820,2012,45515723.0,33764182.0,2420524.0,10259505.0,11751541.0,9612748.0,29843038.0,...,0.0,507788.0,22678546.0,21846600.0,919.0,379207.0,594065.0,-1768573.0,제조업,UQ
3,(주)CMG제약,058820,2013,56997644.0,40742605.0,4848894.0,11327559.0,16255039.0,10688791.0,7997418.0,...,113308.0,1103599.0,43257754.0,33012770.0,832.0,235350.0,87984.0,-3640506.0,제조업,UQ
4,(주)CMG제약,058820,2014,57646794.0,28706417.0,4802462.0,13343937.0,28940377.0,18912799.0,8398498.0,...,158383.0,89925.0,43313433.0,43313433.0,1003.0,172125.0,113135.0,93111.0,제조업,UQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13744,흥구석유(주),024060,2017,79097665.0,14997406.0,2433279.0,5249730.0,64100260.0,37881308.0,10285654.0,...,0.0,0.0,1500000.0,1500000.0,0.0,0.0,0.0,1163482.0,도매 및 소매업,UQ
13745,흥구석유(주),024060,2018,77001426.0,12707368.0,3107658.0,8168930.0,64294058.0,37836314.0,7865391.0,...,0.0,83749.0,1500000.0,1500000.0,0.0,0.0,0.0,-289684.0,도매 및 소매업,UQ
13746,흥구석유(주),024060,2019,82746159.0,21832487.0,2186636.0,3812630.0,60913672.0,34913731.0,6572966.0,...,0.0,0.0,1500000.0,1500000.0,0.0,0.0,0.0,1238029.0,도매 및 소매업,UQ
13747,흥구석유(주),024060,2020,81242681.0,25107354.0,1869624.0,2579206.0,56135327.0,30211463.0,5596818.0,...,0.0,0.0,1500000.0,1500000.0,0.0,0.0,0.0,3685255.0,도매 및 소매업,UQ


In [ ]:
df_industry_op.isna().sum()
# 산업분류,감사의견 뒤에서 필요하면 추가할 예정

## 3.관리종목 가져오기
### 3-1) 관리종목 연마다 가져오기

In [51]:
import pandas as pd

In [147]:
# 1) '관리종목지정' 제목 목록 가져오기
관리종목지정_2011_1 = pd.read_html('./data2/관리종목지정_2011_1.xls',encoding='euc-kr')[0]
관리종목지정_2011_2 = pd.read_html('./data2/관리종목지정_2011_2.xls',encoding='euc-kr')[0]
관리종목지정_2012_2014_1 = pd.read_html('./data2/관리종목지정_2012_2014_1.xls',encoding='euc-kr')[0]
관리종목지정_2012_2014_2 = pd.read_html('./data2/관리종목지정_2012_2014_2.xls',encoding='euc-kr')[0]
관리종목지정_2012_2014_3 = pd.read_html('./data2/관리종목지정_2012_2014_3.xls',encoding='euc-kr')[0]
관리종목지정_2012_2014_4 = pd.read_html('./data2/관리종목지정_2012_2014_4.xls',encoding='euc-kr')[0]
관리종목지정_2015_2017_1 = pd.read_html('./data2/관리종목지정_2015_2017_1.xls',encoding='euc-kr')[0]
관리종목지정_2015_2017_2 = pd.read_html('./data2/관리종목지정_2015_2017_2.xls',encoding='euc-kr')[0]
관리종목지정_2015_2017_3 = pd.read_html('./data2/관리종목지정_2015_2017_3.xls',encoding='euc-kr')[0]
관리종목지정_2018_2020_1 = pd.read_html('./data2/관리종목지정_2018_2020_1.xls',encoding='euc-kr')[0]
관리종목지정_2018_2020_2 = pd.read_html('./data2/관리종목지정_2018_2020_2.xls',encoding='euc-kr')[0]
관리종목지정_2018_2020_3 = pd.read_html('./data2/관리종목지정_2018_2020_3.xls',encoding='euc-kr')[0]
관리종목지정_2018_2020_4 = pd.read_html('./data2/관리종목지정_2018_2020_4.xls',encoding='euc-kr')[0]
관리종목지정_2018_2020_5 = pd.read_html('./data2/관리종목지정_2018_2020_5.xls',encoding='euc-kr')[0]
관리종목지정_2018_2020_6 = pd.read_html('./data2/관리종목지정_2018_2020_6.xls',encoding='euc-kr')[0]

In [148]:
# 2) 관리종목 모두 행 기준으로 합치기
관리종목 = pd.concat([관리종목지정_2011_1,관리종목지정_2011_2,관리종목지정_2012_2014_1,관리종목지정_2012_2014_2,관리종목지정_2012_2014_3,관리종목지정_2012_2014_4,관리종목지정_2015_2017_1,관리종목지정_2015_2017_2,관리종목지정_2015_2017_3,관리종목지정_2018_2020_1,관리종목지정_2018_2020_2,관리종목지정_2018_2020_3,관리종목지정_2018_2020_4,관리종목지정_2018_2020_5,관리종목지정_2018_2020_6],axis=0)
관리종목 = 관리종목[~(관리종목['회사명'].str.contains('스팩'))]
관리종목 = 관리종목[['시간','회사명','종목코드','공시제목']]
관리종목.reset_index(drop=True,inplace=True)
관리종목

,시간,회사명,종목코드,공시제목
0,2011-12-27 18:10,씨티엘테크,88960,관리종목지정사유추가(상장폐지사유발생)
1,2011-12-23 18:32,아큐텍,13780,관리종목지정(파산신청)
2,2011-12-20 16:59,엘앤씨피,15390,관리종목지정(파산신청)
3,2011-12-19 18:34,엔하이테크,46720,관리종목지정(상장폐지사유 발생)
4,2011-12-16 18:23,에이원마이크로,37380,관리종목지정사유추가(상장폐지사유 발생)
...,...,...,...,...
1334,2018-01-31 18:49,에스마크,30270,관리종목지정사유추가(파산신청)
1335,2018-01-26 19:17,에스엘바이오닉스,214310,관리종목지정사유일부해제(상장폐지 사유 해소)
1336,2018-01-26 19:17,에이루트,96690,관리종목지정사유일부해제(상장폐지 사유 해소)
1337,2018-01-22 18:46,위노바,39790,관리종목지정사유추가(상장폐지사유 발생)


In [149]:
# 3) 관리종목 전처리
관리종목['종목코드'] = 관리종목['종목코드'].astype('str')
관리종목['종목코드'] = 관리종목['종목코드'].str.zfill(6)
관리종목.reset_index(drop=True,inplace=True)
관리종목

,시간,회사명,종목코드,공시제목
0,2011-12-27 18:10,씨티엘테크,088960,관리종목지정사유추가(상장폐지사유발생)
1,2011-12-23 18:32,아큐텍,013780,관리종목지정(파산신청)
2,2011-12-20 16:59,엘앤씨피,015390,관리종목지정(파산신청)
3,2011-12-19 18:34,엔하이테크,046720,관리종목지정(상장폐지사유 발생)
4,2011-12-16 18:23,에이원마이크로,037380,관리종목지정사유추가(상장폐지사유 발생)
...,...,...,...,...
1334,2018-01-31 18:49,에스마크,030270,관리종목지정사유추가(파산신청)
1335,2018-01-26 19:17,에스엘바이오닉스,214310,관리종목지정사유일부해제(상장폐지 사유 해소)
1336,2018-01-26 19:17,에이루트,096690,관리종목지정사유일부해제(상장폐지 사유 해소)
1337,2018-01-22 18:46,위노바,039790,관리종목지정사유추가(상장폐지사유 발생)


In [150]:
# 4) 시간을 datetime으로 바꾸기
관리종목['시간'] = pd.to_datetime(관리종목['시간'], infer_datetime_format=True)
관리종목['연도'] = 관리종목['시간'].dt.year

In [151]:
# 5) 연도별로 관리종목 지정 데이터 분리
# years = 관리종목.groupby('연도')
for i in 관리종목['연도'].unique():
    globals()['kosdaq__{}'.format(i)] = 관리종목[관리종목['연도'] == i].sort_values('시간')
    globals()['kosdaq__{}'.format(i)].reset_index(drop=True,inplace=True)

In [152]:
# 6) 무엇을 제거할 건지 확인하는 데이터 프레임 만들기
_2011 = pd.pivot_table(kosdaq__2011, index = ['회사명','종목코드','시간'],values = ['공시제목'], aggfunc='sum')
# -> 각 종목코드별 첫번째거로 해도 상관 없음
# 3) 관리종목 데이터와 산업분류 데이터 merge
for i in 관리종목['연도'].unique():
    globals()['kosdaq__{}'.format(i)].drop_duplicates(['종목코드'],keep='first',inplace=True)
    globals()['kosdaq__{}'.format(i)].reset_index(drop=True,inplace=True)

In [153]:
# 7) 개수 확인
print(len(kosdaq__2011),len(kosdaq__2012),len(kosdaq__2013),len(kosdaq__2014),len(kosdaq__2015),len(kosdaq__2016),len(kosdaq__2017),len(kosdaq__2018),len(kosdaq__2019),len(kosdaq__2020))
print(len(kosdaq__2011)+len(kosdaq__2012)+len(kosdaq__2013)+len(kosdaq__2014)+len(kosdaq__2015)+len(kosdaq__2016)+len(kosdaq__2017)+len(kosdaq__2018)+len(kosdaq__2019)+len(kosdaq__2020))

77 72 56 43 46 43 52 65 94 105
653


In [154]:
# 8) 관리종목 합치기
df_admin = pd.DataFrame()
for i in 관리종목['연도'].unique():
    df_admin_1 = globals()['kosdaq__{}'.format(i)]
    df_admin = pd.concat([df_admin,df_admin_1])
    df_admin.sort_values('연도',inplace=True)
    df_admin.reset_index(drop=True,inplace=True)

In [155]:
df_admin

,시간,회사명,종목코드,공시제목,연도
0,2011-01-14 17:40:00,에스브이에이치,046240,관리종목지정사유추가,2011
1,2011-03-24 07:07:00,케이에스알,066340,관리종목지정(자본잠식률 50% 이상),2011
2,2011-03-22 07:40:00,제일창투,026540,관리종목지정,2011
3,2011-01-18 19:02:00,대양글로벌,040180,관리종목지정사유일부해제,2011
4,2011-02-09 07:08:00,디에이치패션,045260,관리종목지정사유일부해제(파산 신청 사유 해소),2011
...,...,...,...,...,...
648,2020-03-23 19:47:00,한국코퍼레이션,050540,관리종목지정(상장폐지사유 발생),2020
649,2020-03-23 19:29:00,팍스넷,038160,관리종목지정(상장폐지사유 발생 ),2020
650,2020-03-23 19:00:00,엔시트론,101400,관리종목지정(최근3사업연도중 2사업연도 자기자본 50%초과 법인세비용차감전계속사업손...,2020
651,2020-03-27 18:31:00,와이오엠,066430,관리종목지정(최근3사업연도중 2사업연도 자기자본 50%초과 법인세비용차감전계속사업손...,2020


In [156]:
# 9) 관리종목 y df
df_admin = df_admin[['종목코드','연도']]
df_admin.columns = ['거래소코드','회계년도']
df_admin['y'] = 1

# 10) df_admin에 관리종목 지정
df_final = df_industry_op.merge(df_admin,on=['거래소코드','회계년도'],how='left')
df_final['y'] = df_final['y'].fillna(0)
df_final

,회사명,거래소코드,회계년도,자산,유동자산,재고자산,매출채권및기타유동채권,비유동자산,유형자산,부채,...,이자비용,기말자본금,기초자본금,유무형자산등상각비,대손상각비,감가상각비,영업활동으로인한현금흐름,산업분류,감사의견코드,y
0,(주)CMG제약,058820,2010,23456619.0,9965208.0,1946644.0,5063612.0,13491411.0,9055560.0,15723286.0,...,972784.0,10914543.0,8292165.0,1321.0,11516.0,1065106.0,-1388441.0,NaN,NaN,0.0
1,(주)CMG제약,058820,2011,29477750.0,14760027.0,3352328.0,8131871.0,14717723.0,9910510.0,9461282.0,...,0.0,21846600.0,20588188.0,1020.0,0.0,678655.0,-676550.0,제조업,UQ,0.0
2,(주)CMG제약,058820,2012,45515723.0,33764182.0,2420524.0,10259505.0,11751541.0,9612748.0,29843038.0,...,507788.0,22678546.0,21846600.0,919.0,379207.0,594065.0,-1768573.0,제조업,UQ,0.0
3,(주)CMG제약,058820,2013,56997644.0,40742605.0,4848894.0,11327559.0,16255039.0,10688791.0,7997418.0,...,1103599.0,43257754.0,33012770.0,832.0,235350.0,87984.0,-3640506.0,제조업,UQ,0.0
4,(주)CMG제약,058820,2014,57646794.0,28706417.0,4802462.0,13343937.0,28940377.0,18912799.0,8398498.0,...,89925.0,43313433.0,43313433.0,1003.0,172125.0,113135.0,93111.0,제조업,UQ,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13744,흥구석유(주),024060,2017,79097665.0,14997406.0,2433279.0,5249730.0,64100260.0,37881308.0,10285654.0,...,0.0,1500000.0,1500000.0,0.0,0.0,0.0,1163482.0,도매 및 소매업,UQ,0.0
13745,흥구석유(주),024060,2018,77001426.0,12707368.0,3107658.0,8168930.0,64294058.0,37836314.0,7865391.0,...,83749.0,1500000.0,1500000.0,0.0,0.0,0.0,-289684.0,도매 및 소매업,UQ,0.0
13746,흥구석유(주),024060,2019,82746159.0,21832487.0,2186636.0,3812630.0,60913672.0,34913731.0,6572966.0,...,0.0,1500000.0,1500000.0,0.0,0.0,0.0,1238029.0,도매 및 소매업,UQ,0.0
13747,흥구석유(주),024060,2020,81242681.0,25107354.0,1869624.0,2579206.0,56135327.0,30211463.0,5596818.0,...,0.0,1500000.0,1500000.0,0.0,0.0,0.0,3685255.0,도매 및 소매업,UQ,0.0


## 4. 데이터 값 전처리

In [157]:
# 1) 전체 재무제표가 없다면 -> 상장폐지/합병/코스닥 상장 전/코스피로 상장 이전
df_final.dropna(subset=['자산'],axis=0,inplace=True)

In [158]:
df_final.isna().sum()

회사명                0
거래소코드              0
회계년도               0
자산                 0
유동자산               0
재고자산               0
매출채권및기타유동채권        0
비유동자산              0
유형자산               0
부채                 0
유동부채               0
장기차입금              0
단기차입금              0
장기사채               0
단기사채               0
유동성장기부채            0
자본                 0
매출원가               0
매출액                0
당기순이익              0
판매비와관리비            0
(정상)영업손익           0
법인세비용차감전순이익        0
급여                 0
세금과공과              0
이자비용               0
기말자본금              0
기초자본금              0
유무형자산등상각비          0
대손상각비              0
감가상각비              0
영업활동으로인한현금흐름       0
산업분류            1598
감사의견코드          1598
y                  0
dtype: int64

In [159]:
# 1) 자산=0인 코웰이홀딩스유한공사 -> 지주사 & 전부 0으로 나옴 삭제
df_final.drop(df_industry_op[df_industry_op['자산']==0].index,inplace=True)
df_final.reset_index(drop=True,inplace=True)

# 2. 파생변수 만들기

In [65]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

## 2-1) 안정성 변수

In [160]:
df_final_1 = df_final.copy()

In [161]:
# 1. 자기자본비율 = 자본/((기초자산+기말자산)/2)*100 -> (-)/(-) 없음
df_final_1['자기자본비율'] = df_final_1['자본'] /((df_final_1.groupby('거래소코드')['자산'].shift(1) + df_final_1['자산']) / 2) * 100
# 2. 부채비율 = 부채/자본*100 -> (-)/(-) 없음
df_final_1['부채비율'] = df_final_1['부채'] / df_final_1['자본'] * 100
# 3. 유동비율 = 유동자산/유동부채*100 -> (-)/(-) 없음
df_final_1['유동비율'] = df_final_1['유동자산'] / df_final_1['유동부채'] * 100
# 4. 당좌비율 = (유동자산-재고자산)/유동자산*100 -> (-)/(-) 없음
df_final_1['당좌비율'] = (df_final_1['유동자산'] - df_final_1['재고자산']) / df_final_1['유동자산'] * 100
# 5. 차입금의존도 = (장기차입금+단기차입금)/((기초자산+기말자산)/2)*100 -> (-)/(-) 없음
df_final_1['차입금의존도'] = (df_final_1['장기차입금'] + df_final_1['단기차입금']) / ((df_final_1.groupby('거래소코드')['자산'].shift(1) + df_final_1['자산'])/2) * 100
# 6. 매출액규모 = log(매출액) -> <0인경우 없음
df_final_1['매출액규모'] = np.log(df_final_1['매출액'])
# 7. 총자산규모 = log(자산) -> 0 존재 -> 아래 이유로 삭제
df_final_1['총자산규모'] = np.log(df_final_1['자산'])
# 8. 부채상환계수 = (당기순이익+유무형자산 등 상각비(요약-백만원 단위)+이자비용)/(단기차입금+유동성장기부채+이자비용)*100 # -> (-)/(-) 있음 : 듀오벡 -> 12072임 -> 이상치 처리하면서 처리할 예정
df_final_1['부채상환계수'] = (df_final_1['당기순이익']+(df_final_1['유무형자산등상각비'] * 1000)+df_final_1['이자비용']) / (df_final_1['단기차입금']+df_final_1['유동성장기부채']+df_final_1['이자비용']) * 100

In [162]:
df_final_1[df_final_1['자기자본비율'].isin([-np.inf,np.inf])]

,회사명,거래소코드,회계년도,자산,유동자산,재고자산,매출채권및기타유동채권,비유동자산,유형자산,부채,...,감사의견코드,y,자기자본비율,부채비율,유동비율,당좌비율,차입금의존도,매출액규모,총자산규모,부채상환계수


In [163]:
# 8번 (-)/(-) -> min값
df_final_1.loc[((df_final_1['당기순이익']+(df_final_1['유무형자산등상각비'] * 1000)+df_final_1['이자비용'])<0)&((df_final_1['단기차입금']+df_final_1['유동성장기부채']+df_final_1['이자비용'])<0),'부채상환계수']= min(df_final_1['부채상환계수']) 

In [164]:
df_final_1[((df_final_1['당기순이익']+(df_final_1['유무형자산등상각비'] * 1000)+df_final_1['이자비용'])<0)&((df_final_1['단기차입금']+df_final_1['유동성장기부채']+df_final_1['이자비용'])<0)]

,회사명,거래소코드,회계년도,자산,유동자산,재고자산,매출채권및기타유동채권,비유동자산,유형자산,부채,...,감사의견코드,y,자기자본비율,부채비율,유동비율,당좌비율,차입금의존도,매출액규모,총자산규모,부채상환계수
1079,(주)듀오백,073190,2016,38979231.0,15349427.0,4159207.0,4071059.0,23629804.0,9698197.0,5755957.0,...,UQ,0.0,81.202546,17.325075,294.150235,72.903177,1.051545,17.065388,17.47854,-inf


## 2-2) 수익성 변수

In [165]:
# 1. 총자본순이익률 = 당기순이익/((기초자산+기말자산)/2)*100 -> (-)/(-) 없음
df_final_1['총자본순이익률'] = df_final_1['당기순이익']/((df_final_1['자산'] + df_final_1.groupby('거래소코드')['자산'].shift(1))/2) * 100
# 2. 자본금세전순이익률 = 법인세비용차감전순이익 / ((기초자산 + 기말자산)/2)*100 -> (-)/(-) 없음
df_final_1['자본금세전순이익률'] = df_final_1['법인세비용차감전순이익']/((df_final_1.groupby('거래소코드')['자산'].shift(1) + df_final_1['자산'])/2) * 100
# 3. 자기자본순이익률 = 당기순이익 / 자본*100 -> (-) 있음 -> 완전 자본잠식 -> 극단치 대체 -> 이상치 처리(winsorizing) 날리기
df_final_1['자기자본순이익률'] = df_final_1['당기순이익'] / df_final_1['자본'] * 100
# 4. 매출액세전순이익률 = 당기순이익 / 매출액 * 100 -> (-)/(-) 없음
df_final_1['매출액세전순이익률'] = df_final_1['당기순이익'] / df_final_1['매출액'] * 100
# 5. 매출액영업이익률 = (정상)영업손익 / 매출액*100 -> (-)/(-) 없음
df_final_1['매출액영업이익률'] = df_final_1['(정상)영업손익'] / df_final_1['매출액'] * 100
# 6. 금융비용대비매출액 = 이자비용 / 매출액*100 -> (-)/(-) 없음
df_final_1['금융비용대비매출액'] = df_final_1['이자비용'] / df_final_1['매출액'] * 100
# 7. 매출원가율 = 매출원가 / 매출액*100 -> (-)/(-) 없음
df_final_1['매출원가율'] = df_final_1['매출원가'] / df_final_1['매출액'] * 100
# 8. 이자보상배율 = (정상)영업손익/ 이자비용*100 -> (-)/(-) 있음
df_final_1['이자보상배율'] = (df_final_1['(정상)영업손익']) / df_final_1['이자비용'] * 100

In [166]:
# 3번 (-)/(-) -> min값
df_final_1['자기자본순이익률'].describe()

count    13748.000000
mean        -1.794733
std        201.125605
min      -6691.365964
25%         -3.150594
50%          3.264322
75%          8.445493
max      20020.868490
Name: 자기자본순이익률, dtype: float64

In [167]:
# 8번 (-)/(-) -> min값
df_final_1.loc[((df_final_1['(정상)영업손익']<0)&(df_final_1['이자비용']<0)),'이자보상배율'] = min(df_final_1['이자보상배율'])

In [168]:
df_final_1.loc[(df_final_1['자본']<0),'자기자본순이익률'] = min(df_final_1['자기자본순이익률'])

In [76]:
df_final_1[(df_final_1['자본']<0)]['자기자본순이익률']

84      -6691.365964
513     -6691.365964
1322    -6691.365964
2126    -6691.365964
2545    -6691.365964
2913    -6691.365964
3009    -6691.365964
3251    -6691.365964
3640    -6691.365964
3932    -6691.365964
4219    -6691.365964
5284    -6691.365964
5295    -6691.365964
5417    -6691.365964
5574    -6691.365964
6379    -6691.365964
8062    -6691.365964
8764    -6691.365964
9389    -6691.365964
10086   -6691.365964
10087   -6691.365964
10131   -6691.365964
10474   -6691.365964
11540   -6691.365964
11683   -6691.365964
12067   -6691.365964
12420   -6691.365964
12769   -6691.365964
12861   -6691.365964
12913   -6691.365964
13663   -6691.365964
13664   -6691.365964
Name: 자기자본순이익률, dtype: float64

## 2-3) 활동성변수

In [169]:
# 1. 총자본회전율 = 매출액/((기초자산 + 기말자산)/2)*100 -> (-)/(-) 없음
df_final_1['총자본회전율'] = df_final_1['매출액']/((df_final_1.groupby('거래소코드')['자산'].shift(1) + df_final_1['자산'])/2)*100
# 2. 자기자본회전율 = 매출액/자본*100 -> (-)/(-) 없음
df_final_1['자기자본회전율'] = df_final_1['매출액'] / df_final_1['자본'] * 100
# 3. 비유동자산회전율 = 매출액/((비유동자산.shift(1)+비유동자산)/2)*100 -> (-)/(-) 없음
df_final_1['비유동자산회전율'] = df_final_1['매출액']/((df_final_1.groupby('거래소코드')['비유동자산'].shift(1)+df_final_1['비유동자산'])/2)*100
# 4. 유형고정자산회전율 = 매출액/(유형자산.shift(1)+유형자산)/2)*100 -> (-)/(-) 없음
df_final_1['유형고정자산회전율'] = df_final_1['매출액']/((df_final_1.groupby('거래소코드')['유형자산'].shift(1)+df_final_1['유형자산'])/2)*100
# 5. 매출채권회전율 = 매출액/(매출채권.shift(1)+매출채권)/2)*100 -> (-)/(-) 없음
df_final_1['매출채권회전율'] = df_final_1['매출액']/((df_final_1.groupby('거래소코드')['매출채권및기타유동채권'].shift(1)+df_final_1['매출채권및기타유동채권'])/2)*100
# 6. 재고자산회전율 = 매출액/재고자산*100 -> (-)/(-) 없음
df_final_1['재고자산회전율'] = df_final_1['매출액']/df_final_1['재고자산']*100
# 7. 운전자산회전율 = 매출액/(유동자산-유동부채)*100 -> (-)/(-) 없음
df_final_1['운전자산회전율'] = df_final_1['매출액']/(df_final_1['유동자산']-df_final_1['유동부채'])*100

In [78]:
df_final_1[['회사명','거래소코드','회계년도','총자본회전율','자기자본회전율','비유동자산회전율','유형고정자산회전율','매출채권회전율','재고자산회전율','운전자산회전율']]

,회사명,거래소코드,회계년도,총자본회전율,자기자본회전율,비유동자산회전율,유형고정자산회전율,매출채권회전율,재고자산회전율,운전자산회전율
0,(주)CMG제약,058820,2010,NaN,103.196965,NaN,NaN,NaN,409.965356,-661.532225
1,(주)CMG제약,058820,2011,46.384431,61.332767,87.040269,129.459113,186.073568,366.212763,146.669141
2,(주)CMG제약,058820,2012,32.985391,78.917205,93.455149,126.704723,134.502660,510.982126,47.927974
3,(주)CMG제약,058820,2013,29.627408,30.991750,108.446137,149.604688,140.695622,313.185378,44.257043
4,(주)CMG제약,058820,2014,29.637160,34.495971,75.178766,114.782199,137.719075,353.749348,77.520482
...,...,...,...,...,...,...,...,...,...,...
13738,흥구석유(주),024060,2017,145.496858,166.817932,178.848105,302.549984,2346.956843,4717.534405,1850.608041
13739,흥구석유(주),024060,2018,159.716610,180.308414,194.180070,329.271006,1857.981170,4011.319392,2062.242565
13740,흥구석유(주),024060,2019,137.674531,144.362951,175.653482,302.311480,1835.585166,5028.997465,648.082242
13741,흥구석유(주),024060,2020,102.844383,111.475567,144.087785,258.967843,2638.573799,4510.353686,389.875534


## 2-4) 성장성 변수

In [170]:
# 1. 총자산증가율 = (당기말총자본 / 전기말총자본) - 1 -> (-)/(-) 없음
# df_final_1['총자산증가율'] = ((df_final_1['자산'] / df_final_1.groupby('거래소코드')['자산'].shift(1)) - 1) * 100
df_final_1['총자산증가율'] = ((df_final_1['자산'] - df_final_1.groupby('거래소코드')['자산'].shift(1)) / df_final_1.groupby('거래소코드')['자산'].shift(1)) * 100
# 2. 유형자산증가율 = (당기말유형고정자산 / 전기말유형고정자산) - 1 -> (-)/(-) 없음
# df_final_1['유형자산증가율'] = ((df_final_1['유형자산'] / df_final_1.groupby('거래소코드')['유형자산'].shift(1)) - 1) * 100
df_final_1['유형자산증가율'] = ((df_final_1['유형자산'] - df_final_1.groupby('거래소코드')['유형자산'].shift(1)) / df_final_1.groupby('거래소코드')['유형자산'].shift(1)) * 100
# 3. 매출액증가율 = (당기말매출액 / 전기말매출액) - 1 -> (-)/(-) 없음
# df_final_1['매출액증가율'] = ((df_final_1['매출액'] / df_final_1.groupby('거래소코드')['매출액'].shift(1)) - 1) * 100
df_final_1['매출액증가율'] = ((df_final_1['매출액'] - df_final_1.groupby('거래소코드')['매출액'].shift(1)) / df_final_1.groupby('거래소코드')['매출액'].shift(1)) * 100
# 4. 순이익증가율 = (당기말순이익 / 전기말순이익) - 1 -> (-)/(-) 없음
# df_final_1['순이익증가율'] = ((df_final_1['당기순이익'] / df_final_1.groupby('거래소코드')['당기순이익'].shift(1)) - 1) * 100
df_final_1['순이익증가율'] = ((df_final_1['당기순이익'] - df_final_1.groupby('거래소코드')['당기순이익'].shift(1))/ df_final_1.groupby('거래소코드')['당기순이익'].shift(1)) * 100
# 5. 자기자본증가율 = (당기말자기자본 / 전기말자기자본) - 1 -> (-)/(-) 없음
# df_final_1['자기자본증가율'] = ((df_final_1['자본'] / df_final_1.groupby('거래소코드')['자본'].shift(1)) - 1) * 100
df_final_1['자기자본증가율'] = ((df_final_1['자본'] - df_final_1.groupby('거래소코드')['자본'].shift(1))/ df_final_1.groupby('거래소코드')['자본'].shift(1)) * 100

4번 수정

In [80]:
df_final_1['순이익증가율'].describe()

count    1.203400e+04
mean     7.481944e+01
std      2.311714e+04
min     -5.098021e+05
25%     -9.334582e+01
50%     -1.799029e+01
75%      4.727778e+01
max      2.400869e+06
Name: 순이익증가율, dtype: float64

In [171]:
# 극단치 처리
df_final_1.loc[(df_final_1['당기순이익']<0)&(df_final_1.groupby('거래소코드')['당기순이익'].shift(1)<0),'순이익증가율'] = np.nanmin(df_final_1['순이익증가율'])

5번

In [83]:
df_final_1['자기자본증가율'].describe()

count    12034.000000
mean         9.912563
std        695.753387
min     -73839.057304
25%         -2.629811
50%          6.022475
75%         17.799140
max      12506.498549
Name: 자기자본증가율, dtype: float64

In [172]:
# 극단치 처리
df_final_1.loc[(df_final_1['자본']<0)&(df_final_1.groupby('거래소코드')['자본'].shift(1)<0),'자기자본증가율'] = np.nanmin(df_final_1['자기자본증가율'])

(-) -> (+) 수정

In [173]:
# 순이익증가율 계산
value = []
for i in range(len(df_final_1)):
    if ((df_final_1['당기순이익']>0) & (df_final_1.groupby('거래소코드')['당기순이익'].shift(1)<0))[i] :
        value1 = (1-(df_final_1.groupby('거래소코드')['당기순이익'].shift(1)[i] / df_final_1['당기순이익'][i])) * 100
        value.append(value1)
    elif ((df_final_1['당기순이익']<0) & (df_final_1.groupby('거래소코드')['당기순이익'].shift(1)<0))[i] :
        value1 = ((df_final_1['당기순이익'][i] / abs(df_final_1.groupby('거래소코드')['당기순이익'].shift(1)[i])) - 1) * 100
        value.append(value1)
    else:
        value1 =((df_final_1['당기순이익'][i] / df_final_1.groupby('거래소코드')['당기순이익'].shift(1)[i]) - 1) * 100
        value.append(value1)
df_final_1['순이익증가율'] = pd.DataFrame(value,columns=['순이익증가율'])

In [174]:
# 자기자본증가율 계산
value = []
for i in range(len(df_final_1)):
    if ((df_final_1['자본']>0) & (df_final_1.groupby('거래소코드')['자본'].shift(1)<0))[i] :
        value1 = (1-(df_final_1.groupby('거래소코드')['자본'].shift(1)[i] / df_final_1['자본'][i])) * 100
        value.append(value1)
    elif ((df_final_1['자본']<0) & (df_final_1.groupby('거래소코드')['자본'].shift(1)<0))[i] :
        value1 = ((df_final_1['자본'][i] / abs(df_final_1.groupby('거래소코드')['자본'].shift(1)[i])) - 1) * 100
        value.append(value1)
    else:
        value1 =((df_final_1['자본'][i] / df_final_1.groupby('거래소코드')['자본'].shift(1)[i]) - 1) * 100
        value.append(value1)
df_final_1['자기자본증가율'] = pd.DataFrame(value,columns=['자기자본증가율'])

In [80]:
df_final_1[['회사명','거래소코드','총자산증가율','유형자산증가율','매출액증가율','순이익증가율','자기자본증가율']]

,회사명,거래소코드,총자산증가율,유형자산증가율,매출액증가율,순이익증가율,자기자본증가율
0,(주)CMG제약,058820,NaN,NaN,NaN,NaN,NaN
1,(주)CMG제약,058820,25.669219,9.441161,53.831859,831.580106,158.833603
2,(주)CMG제약,058820,54.407046,-3.004507,0.747696,-687.677616,-21.701042
3,(주)CMG제약,058820,25.226274,11.193917,22.780406,720.967790,212.647290
4,(주)CMG제약,058820,1.138907,76.940488,11.870458,-50.813199,0.506265
...,...,...,...,...,...,...,...
13738,흥구석유(주),024060,0.513277,-0.314656,9.579261,-21.850091,0.541185
13739,흥구석유(주),024060,-2.650191,-0.118776,8.595912,-7.191638,0.470883
13740,흥구석유(주),024060,7.460554,-7.724280,-11.786013,654.093800,10.178712
13741,흥구석유(주),024060,-1.816976,-13.468248,-23.315611,-88.954713,-0.692278


## 2-5) 생산성변수

In [175]:
# 1. 총자본투자효율 = (급여 +  (정상)영업손익 + 대손상각비 - 이자비용+ 세금과 공과 + 감가상각비 + 이자비용) / ((기초자본금 + 기말자본금)/2)*100
# 부가가치 = 인건비 + 영업잉여** + 세금과 공과 + 감가상각비 + 이자비용
# 영업잉여 = 영업손익 + 대손상각비 - 이자비용
df_final_1['총자본투자효율'] = (df_final_1['급여'] + df_final_1['(정상)영업손익'] + df_final_1['대손상각비'] - df_final_1['이자비용'] + df_final_1['세금과공과'] + df_final_1['감가상각비'] + df_final_1['이자비용']) / ((df_final_1.groupby('거래소코드')['자산'].shift(1) + df_final_1['자산'])/2) * 100
# 2. 부가가치율 = (급여 +  (정상)영업손익 + 대손상각비 - 이자비용+ 세금과 공과 + 감가상각비 + 이자비용) / 매출액*100
df_final_1['부가가치율'] = (df_final_1['급여'] + df_final_1['(정상)영업손익'] + df_final_1['대손상각비'] - df_final_1['이자비용'] + df_final_1['세금과공과'] + df_final_1['감가상각비'] + df_final_1['이자비용']) / df_final_1['매출액'] * 100
# 3. 노동소득분배율 = 급여 / (급여 +  (정상)영업손익 + 대손상각비 - 이자비용+ 세금과 공과 + 감가상각비 + 이자비용)*100
df_final_1['노동소득분배율'] = df_final_1['급여'] / (df_final_1['급여'] + df_final_1['(정상)영업손익'] + df_final_1['대손상각비'] - df_final_1['이자비용'] + df_final_1['세금과공과'] + df_final_1['감가상각비'] + df_final_1['이자비용']) * 100

In [82]:
df_final_1[['회사명','거래소코드','총자본투자효율','부가가치율','노동소득분배율']]

,회사명,거래소코드,총자본투자효율,부가가치율,노동소득분배율
0,(주)CMG제약,058820,-7.792038,-9.376490,-239.384763
1,(주)CMG제약,058820,15.760847,27.239028,56.204155
2,(주)CMG제약,058820,-8.611934,-15.501043,-0.000000
3,(주)CMG제약,058820,9.210143,23.128577,58.087209
4,(주)CMG제약,058820,6.959333,17.743146,76.622734
...,...,...,...,...,...
13738,흥구석유(주),024060,12.787067,0.167092,0.000000
13739,흥구석유(주),024060,3.418800,0.041138,0.000000
13740,흥구석유(주),024060,31.389133,0.428166,0.000000
13741,흥구석유(주),024060,-19.502533,-0.346910,-0.000000


## 2-6)현금흐름

In [176]:
# 현금흐름 = 영업활동으로인한현금흐름 / 자본 * 100
df_final_1['현금흐름'] = df_final_1['영업활동으로인한현금흐름'] / df_final_1['자본'] * 100
df_final_1[['회사명','거래소코드','현금흐름']]

,회사명,거래소코드,현금흐름
0,(주)CMG제약,058820,-17.953977
1,(주)CMG제약,058820,-3.379967
2,(주)CMG제약,058820,-11.284429
3,(주)CMG제약,058820,-7.429570
4,(주)CMG제약,058820,0.189064
...,...,...,...
13743,흥구석유(주),024060,1.690812
13744,흥구석유(주),024060,-0.419006
13745,흥구석유(주),024060,1.625282
13746,흥구석유(주),024060,4.871720


## 2-7) 파생변수 df만 가져오기

In [177]:
df = pd.concat([df_final_1.iloc[:,:3],df_final_1.iloc[:,-35:]],axis=1)

In [179]:
# 임시저장
# df.to_csv('./data/전처리data.csv')

## 2-8) 첫번째 회계년도 삭제

In [129]:
# 1) 각 거래소코드별 회계년도 1번째 값 가져오기
a = df.groupby('거래소코드')['회계년도'].nth(0,dropna=False)

In [ ]:
# 2) 거래소코드와 회계년도가 각 거래소코드별 회계년도 1번째 값과 같으면 회계년도를 np.NaN값으로 만들어서 삭제
for i in range(len(df)):
    print(i+1, "/", len(df))
    for j in range(len(a)):
        if (df['거래소코드'][i] == a.index[j]) & (df['회계년도'][i] == a.iloc[j]):
            df['회계년도'][i]= np.NaN
        else:
            pass

In [156]:
df = df[df['회계년도'].notna()]
df.reset_index(drop=True,inplace=True)
df.to_csv('./data/전처리data_회계년도수정.csv')

In [3]:
import pandas as pd
df = pd.read_csv('./data/전처리data_회계년도수정.csv',index_col=0)
df['회계년도'] = pd.to_datetime(df['회계년도'],format='%Y')
df['회계년도'] = df['회계년도'].dt.year
df

,회사명,거래소코드,회계년도,산업분류,감사의견코드,y,자기자본비율,부채비율,유동비율,당좌비율,...,운전자산회전율,총자산증가율,유형자산증가율,매출액증가율,순이익증가율,자기자본증가율,총자본투자효율,부가가치율,노동소득분배율,현금흐름
0,(주)CMG제약,58820,2011,제조업,UQ,0.0,75.627489,47.267492,230.996351,77.287792,...,146.669141,25.669219,9.441161,53.831859,831.580106,158.833603,12.634668,27.239028,56.204155,-3.379967
1,(주)CMG제약,58820,2012,제조업,UQ,0.0,41.797464,190.414329,424.286941,92.831089,...,47.927974,54.407046,-3.004507,0.747696,-687.677616,-21.701042,-5.113080,-15.501043,-0.000000,-11.284429
2,(주)CMG제약,58820,2013,제조업,UQ,0.0,95.597728,16.321186,633.696024,88.098714,...,44.257043,25.226274,11.193917,22.780406,720.967790,212.647290,6.852398,23.128577,58.087209,-7.429570
3,(주)CMG제약,58820,2014,제조업,UQ,0.0,85.914846,17.053378,422.691875,83.270423,...,77.520482,1.138907,76.940488,11.870458,-50.813199,0.506265,5.258565,17.743146,76.622734,0.189064
4,(주)CMG제약,58820,2015,제조업,UQ,0.0,86.334675,16.442479,358.939268,81.390646,...,100.565231,1.066129,-6.083943,15.520101,-369.911312,1.596356,4.993929,14.747191,85.089072,2.766692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12033,흥구석유(주),24060,2017,도매 및 소매업,UQ,0.0,87.218955,14.947469,170.530914,83.775334,...,1850.608041,0.513277,-0.314656,9.579261,-21.850091,0.541185,0.243113,0.167092,0.000000,1.690812
12034,흥구석유(주),24060,2018,도매 및 소매업,UQ,0.0,88.579677,11.376688,190.727293,75.544440,...,2062.242565,-2.650191,-0.118776,8.595912,-7.191638,0.470883,0.065704,0.041138,0.000000,-0.419006
12035,흥구석유(주),24060,2019,도매 및 소매업,UQ,0.0,95.366942,8.628975,448.803427,89.984485,...,648.082242,7.460554,-7.724280,-11.786013,654.093800,10.178712,0.589476,0.428166,0.000000,1.625282
12036,흥구석유(주),24060,2020,도매 및 소매업,UQ,0.0,92.257330,7.398710,721.842419,92.553481,...,389.875534,-1.816976,-13.468248,-23.315611,-88.954713,-0.692278,-0.356778,-0.346910,-0.000000,4.871720


## 2-9) 결측치 확인

In [4]:
# 1) 재고자산이 0 na값은 삭제
df.dropna(subset=['재고자산회전율'],axis=0,inplace=True)

In [5]:
# 2) 유형자산 0/0꼴 문제 -> 0으로 채우기
df['유형자산증가율'] = df['유형자산증가율'].fillna(0)

In [6]:
df.isna().sum()

회사명          0
거래소코드        0
회계년도         0
산업분류         0
감사의견코드       0
y            0
자기자본비율       0
부채비율         0
유동비율         0
당좌비율         0
차입금의존도       0
매출액규모        0
총자산규모        0
부채상환계수       0
총자본순이익률      0
자본금세전순이익률    0
자기자본순이익률     0
매출액세전순이익률    0
매출액영업이익률     0
금융비용대비매출액    0
매출원가율        0
이자보상배율       0
총자본회전율       0
자기자본회전율      0
비유동자산회전율     0
유형고정자산회전율    0
매출채권회전율      0
재고자산회전율      0
운전자산회전율      0
총자산증가율       0
유형자산증가율      0
매출액증가율       0
순이익증가율       0
자기자본증가율      0
총자본투자효율      0
부가가치율        0
노동소득분배율      0
현금흐름         0
dtype: int64

1. 지주회사라고 구분이 되어있는 (주)아시아미디어홀딩스,에스앤씨엔진그룹리미티드 산업분류 수정
- (주)아시아미디어홀딩스 : 정보통신업
- 에스앤씨엔진그룹리미티드 : 전문, 과학 및 기술 서비스업
2. 2019년,2020년 광업 제외

In [7]:
df.loc[df['회사명']=='(주)아시아미디어홀딩스','산업분류']='정보통신업'
df[df['회사명']=='(주)아시아미디어홀딩스']

,회사명,거래소코드,회계년도,산업분류,감사의견코드,y,자기자본비율,부채비율,유동비율,당좌비율,...,운전자산회전율,총자산증가율,유형자산증가율,매출액증가율,순이익증가율,자기자본증가율,총자본투자효율,부가가치율,노동소득분배율,현금흐름
3254,(주)아시아미디어홀딩스,52810,2011,정보통신업,UQ,1.0,39.599363,166.542071,239.680415,100.0,...,13.575628,11.749925,213.759887,-42.761558,-115.801793,27.924157,4.310206,66.958349,45.124549,-13.007287


In [8]:
df.loc[df['회사명']=='에스앤씨엔진그룹리미티드','산업분류']='전문, 과학 및 기술 서비스업'
df[df['회사명']=='에스앤씨엔진그룹리미티드']

,회사명,거래소코드,회계년도,산업분류,감사의견코드,y,자기자본비율,부채비율,유동비율,당좌비율,...,운전자산회전율,총자산증가율,유형자산증가율,매출액증가율,순이익증가율,자기자본증가율,총자본투자효율,부가가치율,노동소득분배율,현금흐름
10124,에스앤씨엔진그룹리미티드,900080,2011,"전문, 과학 및 기술 서비스업",UQ,0.0,91.099767,31.029842,214.491541,89.723459,...,196.789234,48.040112,36.074128,38.732042,9.274723,24.793966,19.396117,30.453921,4.444973,8.756525
10125,에스앤씨엔진그룹리미티드,900080,2012,"전문, 과학 및 기술 서비스업",UQ,0.0,88.978015,14.111073,399.369680,88.129309,...,132.222004,3.115319,19.435117,-5.401791,3.325940,18.403793,13.580835,27.325755,5.200611,8.492105
10126,에스앤씨엔진그룹리미티드,900080,2013,"전문, 과학 및 기술 서비스업",UQ,0.0,90.111287,16.550557,777.344718,91.435603,...,85.582326,10.582190,-1.445540,-6.699329,-37.792444,8.267628,10.467231,24.132088,8.601318,7.254936
10127,에스앤씨엔진그룹리미티드,900080,2014,"전문, 과학 및 기술 서비스업",UQ,0.0,90.783970,9.859854,598.151604,93.676398,...,86.149787,-0.528327,-9.835830,-7.230926,-49.712805,5.529713,9.602728,24.997655,10.319445,11.975513
10128,에스앤씨엔진그룹리미티드,900080,2015,"전문, 과학 및 기술 서비스업",UQ,0.0,100.445955,9.944137,617.147899,94.761645,...,76.860254,23.300114,13.263829,15.087430,166.533502,23.205592,10.737008,27.043765,12.381921,7.416641
10129,에스앤씨엔진그룹리미티드,900080,2016,"전문, 과학 및 기술 서비스업",UQ,0.0,92.318180,11.326012,579.530781,94.502813,...,57.930255,5.706607,1.695721,-16.901570,-16.771295,4.394485,7.933412,27.313278,15.118469,6.690863
10130,에스앤씨엔진그룹리미티드,900080,2017,"전문, 과학 및 기술 서비스업",UQ,0.0,92.993509,10.738887,579.861158,95.005810,...,55.738391,6.143015,52.732950,-2.586699,-19.932784,6.705773,6.661053,24.937963,14.056028,5.271938
10131,에스앤씨엔진그룹리미티드,900080,2018,"전문, 과학 및 기술 서비스업",UQ,0.0,91.847879,4.872358,1196.223610,96.368530,...,44.363659,-7.093117,-5.255705,-19.067412,-15.447510,-1.895933,5.312468,24.409649,12.568639,9.164611
10132,에스앤씨엔진그룹리미티드,900080,2019,"전문, 과학 및 기술 서비스업",UQ,0.0,99.122141,4.801211,1321.938352,96.121156,...,37.550767,8.075848,39.176050,-12.584065,-43.297512,8.149218,4.757429,25.059289,12.676243,5.108499
10133,에스앤씨엔진그룹리미티드,900080,2020,"전문, 과학 및 기술 서비스업",UQ,1.0,93.067534,0.202583,20118.671431,89.003464,...,5.799767,-12.635711,8.963659,-88.778834,-677.037736,-8.626275,-4.882333,-223.038377,-9.484125,-15.579193


In [9]:
df = df.loc[~((df['산업분류']=='광업')&((df['회계년도']==2019)|(df['회계년도']==2020)))]

In [37]:
# 임시저장
df.to_csv('./data/kosdaq_전처리.csv')

# 3. 리샘플링하기

In [27]:
df = pd.read_csv('./data/kosdaq_전처리.csv',index_col=0)

## 3-1) 연도별, 산업별 under 샘플링

In [28]:
df_admin = df[df['y']==1]
df_admin.reset_index(drop=True,inplace=True)

In [30]:
# 1) 연도별로 산업 데이터 분리
for i in df['회계년도'].unique():
    globals()['industry__{}'.format(i)] = df[df['회계년도'] == i]
    globals()['industry__{}'.format(i)].reset_index(drop=True,inplace=True)

In [32]:
# 2) 연도별로  관리종목 지정 데이터 분리
# years = 관리종목.groupby('연도')
for i in df['회계년도'].unique():
    globals()['kosdaq__{}'.format(i)] = df_admin[df_admin['회계년도'] == i]
    globals()['kosdaq__{}'.format(i)].reset_index(drop=True,inplace=True)

In [33]:
# 3) 산업데이터에서 관리종목 빼기
for i in df['회계년도'].unique():
    globals()['kosdaq_not_{}'.format(i)] = globals()['industry__{}'.format(i)][globals()['industry__{}'.format(i)]['거래소코드'].isin(set(globals()['industry__{}'.format(i)]['거래소코드'].tolist()) - set(globals()['kosdaq__{}'.format(i)]['거래소코드'].tolist()))]

In [39]:
# 4) label 빈도수 체크
for i in df['회계년도'].unique():
    globals()['kosdaq__{}_count'.format(i)] = globals()['kosdaq__{}'.format(i)].groupby('산업분류').count()
    globals()['kosdaq__{}_val'.format(i)] = globals()['kosdaq__{}'.format(i)].groupby('산업분류')['거래소코드'].count().tolist()
    globals()['kosdaq__{}_li'.format(i)] = globals()['kosdaq__{}_count'.format(i)].index.tolist()

In [40]:
# 5) 산업분류에 따라 랜덤 뽑고 재 df화하기
df_sample = pd.DataFrame()
for i in df['회계년도'].unique():
    for j,k in zip(globals()['kosdaq__{}_li'.format(i)],globals()['kosdaq__{}_val'.format(i)]):
        try:
            df_sample_1 = globals()['kosdaq_not_{}'.format(i)].loc[globals()['kosdaq_not_{}'.format(i)]['산업분류']==j].sample(n=k)
            df_sample_1 = df_sample_1.reset_index(drop=True)
            df_sample = pd.concat([df_sample,df_sample_1])
            df_sample.reset_index(drop=True,inplace=True)
        except:
            pass

In [16]:
# 임시저장
# df_sample.to_csv('./kosdaq_not.csv')

In [42]:
# 6) count 확인
kosdaq__2020.groupby('산업분류').count()

,회사명,거래소코드,회계년도,감사의견코드,y,자기자본비율,부채비율,유동비율,당좌비율,차입금의존도,...,운전자산회전율,총자산증가율,유형자산증가율,매출액증가율,순이익증가율,자기자본증가율,총자본투자효율,부가가치율,노동소득분배율,현금흐름
산업분류,,,,,,,,,,,,,,,,,,,,,
건설업,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
도매 및 소매업,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14
"사업시설 관리, 사업 지원 및 임대 서비스업",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"예술, 스포츠 및 여가관련 서비스업",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"전문, 과학 및 기술 서비스업",5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
정보통신업,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16
제조업,56,56,56,56,56,56,56,56,56,56,...,56,56,56,56,56,56,56,56,56,56


In [200]:
df_sample.groupby(['회계년도','산업분류']).count()

회사명  거래소코드  감사의견코드   y  자기자본비율  부채비율  유동비율  \
회계년도 산업분류                                                                   
2011 광업                          1      1       1   1       1     1     1   
     도매 및 소매업                    6      6       6   6       6     6     6   
     전문, 과학 및 기술 서비스업            1      1       1   1       1     1     1   
     정보통신업                       3      3       3   3       3     3     3   
     제조업                        24     24      24  24      24    24    24   
2012 건설업                         1      1       1   1       1     1     1   
     광업                          1      1       1   1       1     1     1   
     교육 서비스업                     3      3       3   3       3     3     3   
     도매 및 소매업                    4      4       4   4       4     4     4   
     전문, 과학 및 기술 서비스업            2      2       2   2       2     2     2   
     정보통신업                       3      3       3   3       3     3     3   
     제조업                        24     24      24  24      24    24    24   
2013 건설업                         2      2       2   2       2     2     2   
     도매 및 소매업                    4      4       4   4       4     4     4   
     정보통신업                       3      3       3   3       3     3     3   
     제조업                        18     18      18  18      18    18    18   
2014 건설업                         2      2       2   2       2     2     2   
     교육 서비스업                     1      1       1   1       1     1     1   
     도매 및 소매업                    5      5       5   5       5     5     5   
     수도, 하수 및 폐기물 처리, 원료 재생업     1      1       1   1       1     1     1   
     정보통신업                       4      4       4   4       4     4     4   
     제조업                        17     17      17  17      17    17    17   
2015 건설업                         3      3       3   3       3     3     3   
     도매 및 소매업                    4      4       4   4       4     4     4   
     수도, 하수 및 폐기물 처리, 원료 재생업     1      1       1   1       1     1     1   
     전기, 가스, 증기 및 공기조절 공급업       1      1       1   1       1     1     1   
     전문, 과학 및 기술 서비스업            1      1       1   1       1     1     1   
     정보통신업                       6      6       6   6       6     6     6   
     제조업                        16     16      16  16      16    16    16   
2016 건설업                         2      2       2   2       2     2     2   
     도매 및 소매업                    6      6       6   6       6     6     6   
     수도, 하수 및 폐기물 처리, 원료 재생업     1      1       1   1       1     1     1   
     전기, 가스, 증기 및 공기조절 공급업       1      1       1   1       1     1     1   
     정보통신업                       5      5       5   5       5     5     5   
     제조업                        19     19      19  19      19    19    19   
2017 건설업                         2      2       2   2       2     2     2   
     도매 및 소매업                    9      9       9   9       9     9     9   
     사업시설 관리, 사업 지원 및 임대 서비스업    1      1       1   1       1     1     1   
     전문, 과학 및 기술 서비스업            1      1       1   1       1     1     1   
     정보통신업                       3      3       3   3       3     3     3   
     제조업                        28     28      28  28      28    28    28   
2018 건설업                         1      1       1   1       1     1     1   
     도매 및 소매업                    5      5       5   5       5     5     5   
     예술, 스포츠 및 여가관련 서비스업         2      2       2   2       2     2     2   
     정보통신업                       6      6       6   6       6     6     6   
     제조업                        33     33      33  33      33    33    33   
2019 건설업                         4      4       4   4       4     4     4   
     도매 및 소매업                    7      7       7   7       7     7     7   
     예술, 스포츠 및 여가관련 서비스업         1      1       1   1       1     1     1   
     전문, 과학 및 기술 서비스업            4      4       4   4       4     4     4   
     정보통신업                 

In [17]:
df_final = pd.concat([df_admin,df_sample],axis=0)
df_final.reset_index(drop=True,inplace=True)
df_final

,회사명,거래소코드,회계년도,산업분류,감사의견코드,y,자기자본비율,부채비율,유동비율,당좌비율,...,운전자산회전율,총자산증가율,유형자산증가율,매출액증가율,순이익증가율,자기자본증가율,총자본투자효율,부가가치율,노동소득분배율,현금흐름
0,(주)강원에너지,114190,2020,제조업,UQ,1.0,41.659492,78.715434,171.652368,97.346502,...,260.835838,-40.698452,-7.485894,-10.095643,-146.148683,-36.568135,-3.355146,-6.871342,-99.011783,17.397557
1,(주)골드앤에스,35290,2014,정보통신업,UQ,1.0,20.669768,335.311991,51.192980,78.536538,...,-163.121344,-18.218210,21.664094,-20.337630,-231.855405,-55.327901,2.745650,6.027474,193.362805,-8.818698
2,(주)골드앤에스,35290,2015,정보통신업,UQ,1.0,48.368876,42.093766,324.774250,80.879343,...,155.943863,-47.643241,-98.376233,-37.703368,-239.163255,60.397782,5.770305,13.937739,76.620945,7.409362
3,(주)골드앤에스,35290,2019,정보통신업,DS,1.0,37.021190,171.147017,41.744283,76.596773,...,-58.441035,0.766632,38.002005,-7.731282,-442.614164,-8.762799,-6.899849,-33.839080,-69.583921,-45.108715
4,(주)골드앤에스,35290,2020,정보통신업,UQ,1.0,-12.754704,-704.718468,17.931353,77.878012,...,-17.828586,-37.226271,-61.348826,-47.782566,-140.041210,-128.146832,-0.120078,-0.921375,-2612.069777,17.882626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,(주)바이오솔루션,86820,2020,제조업,UQ,0.0,93.079812,4.890369,2761.424968,96.465210,...,14.103125,-4.626905,45.244686,-22.856771,-326.947283,-2.185597,-0.804465,-7.244845,-337.923284,-4.101015
934,(주)씨유메디칼시스템,115480,2020,제조업,UQ,0.0,59.525673,92.000919,117.374389,75.839411,...,595.979618,33.344563,2.677153,-17.232473,-853.589342,66.091353,3.849428,10.315665,101.143427,8.226030
935,(주)메카로,241770,2020,제조업,UQ,0.0,89.105169,11.156992,629.191005,78.976175,...,92.219919,-1.888743,19.122630,-7.706858,-110.388840,-3.774855,5.070220,15.061557,76.633829,4.800863
936,비디아이(주),148140,2020,제조업,UQ,0.0,32.634844,216.670953,135.740416,100.000000,...,453.226828,6.921679,-2.693640,124.657694,-2427.908560,55.448194,-3.525107,-3.429733,-28.575109,-69.421513


## 3-2. Labelencoding, Onehotencoding

1. label encoding

In [18]:
df_final_1 = df_final.copy()

In [20]:
import numpy as np

In [21]:
# 1) 감사의견코드 UQ -> 1 / 나머지 0
df_final_1['감사의견코드'] = np.where(df_final_1['감사의견코드'] == 'UQ', 1, 0)
df_final_1.head()

,회사명,거래소코드,회계년도,산업분류,감사의견코드,y,자기자본비율,부채비율,유동비율,당좌비율,...,운전자산회전율,총자산증가율,유형자산증가율,매출액증가율,순이익증가율,자기자본증가율,총자본투자효율,부가가치율,노동소득분배율,현금흐름
0,(주)강원에너지,114190,2020,제조업,1,1.0,41.659492,78.715434,171.652368,97.346502,...,260.835838,-40.698452,-7.485894,-10.095643,-146.148683,-36.568135,-3.355146,-6.871342,-99.011783,17.397557
1,(주)골드앤에스,35290,2014,정보통신업,1,1.0,20.669768,335.311991,51.192980,78.536538,...,-163.121344,-18.218210,21.664094,-20.337630,-231.855405,-55.327901,2.745650,6.027474,193.362805,-8.818698
2,(주)골드앤에스,35290,2015,정보통신업,1,1.0,48.368876,42.093766,324.774250,80.879343,...,155.943863,-47.643241,-98.376233,-37.703368,-239.163255,60.397782,5.770305,13.937739,76.620945,7.409362
3,(주)골드앤에스,35290,2019,정보통신업,0,1.0,37.021190,171.147017,41.744283,76.596773,...,-58.441035,0.766632,38.002005,-7.731282,-442.614164,-8.762799,-6.899849,-33.839080,-69.583921,-45.108715
4,(주)골드앤에스,35290,2020,정보통신업,1,1.0,-12.754704,-704.718468,17.931353,77.878012,...,-17.828586,-37.226271,-61.348826,-47.782566,-140.041210,-128.146832,-0.120078,-0.921375,-2612.069777,17.882626


2. Onehotencoding

In [22]:
# 방법 1) 산업분류 -> n-1개의 더미변수 만들기
df_final_1 = pd.concat([df_final_1,pd.get_dummies(df_final_1['산업분류'],drop_first=True)],axis=1)
df_final_1.drop('산업분류',axis=1,inplace=True)
df_final_1.head()

,회사명,거래소코드,회계년도,감사의견코드,y,자기자본비율,부채비율,유동비율,당좌비율,차입금의존도,...,광업,교육 서비스업,도매 및 소매업,"사업시설 관리, 사업 지원 및 임대 서비스업","수도, 하수 및 폐기물 처리, 원료 재생업","예술, 스포츠 및 여가관련 서비스업","전기, 가스, 증기 및 공기조절 공급업","전문, 과학 및 기술 서비스업",정보통신업,제조업
0,(주)강원에너지,114190,2020,1,1.0,41.659492,78.715434,171.652368,97.346502,12.303099,...,0,0,0,0,0,0,0,0,0,1
1,(주)골드앤에스,35290,2014,1,1.0,20.669768,335.311991,51.192980,78.536538,36.337895,...,0,0,0,0,0,0,0,0,1,0
2,(주)골드앤에스,35290,2015,1,1.0,48.368876,42.093766,324.774250,80.879343,0.598762,...,0,0,0,0,0,0,0,0,1,0
3,(주)골드앤에스,35290,2019,0,1.0,37.021190,171.147017,41.744283,76.596773,6.594573,...,0,0,0,0,0,0,0,0,1,0
4,(주)골드앤에스,35290,2020,1,1.0,-12.754704,-704.718468,17.931353,77.878012,6.355326,...,0,0,0,0,0,0,0,0,1,0


In [25]:
# 2) 무한대값확인
np.isinf(abs(df_final_1.iloc[:,5:37])).sum()

자기자본비율       0
부채비율         0
유동비율         0
당좌비율         0
차입금의존도       0
매출액규모        0
총자산규모        0
부채상환계수       0
총자본순이익률      0
자본금세전순이익률    0
자기자본순이익률     0
매출액세전순이익률    0
매출액영업이익률     0
금융비용대비매출액    0
매출원가율        0
이자보상배율       0
총자본회전율       0
자기자본회전율      0
비유동자산회전율     0
유형고정자산회전율    0
매출채권회전율      0
재고자산회전율      0
운전자산회전율      0
총자산증가율       0
유형자산증가율      0
매출액증가율       0
순이익증가율       0
자기자본증가율      0
총자본투자효율      0
부가가치율        0
노동소득분배율      0
현금흐름         0
dtype: int64

>   재무비율 : 1. 부채상환계수(48), 2. 이자보상배율(144), 3.재고자산회전율(66)
>
>   1. 부채상환계수
>   *  inf → 단기차입금, 이자비용, 단기사채, 유동성 장기부채가 없음 -> 부채를 잘 상환한다. max
>   *  -inf → 당기순이익이 음수(-)인 경우 -> 0(모델 학습에 고려하지 않음)
>    
>   2. 이자보상배율(144)
>   *   inf → 이자 비용이 없음 → 이자를 잘 갚는다. max
>   *   -inf → 정상영업손익이 <0 → 0 (모델 학습에 고려하지 않음)
>
>   3. 재고자산회전율 → 0 (모델 학습에 고려하지 않음)
>   * inf → 재고자산이 없는 경우(기술/연구, 서비스, 디자인 등) 
>            또는 재고자산이 없다가 생긴경우(신약개발 등)
>        예외 ) 네오 이뮨텍, 메드팩토, 박셀바이오, 셀리드
>

In [24]:
# 2) inf,-inf 값 대체
## 1) 부채상환계수
## -inf 값은 0으로 대체
df_final_1.loc[df_final_1['부채상환계수'] == -np.inf, '부채상환계수'] = 0
## inf 값은 부채상환계수 중 최대값으로 대체
df_final_1.loc[df_final_1['부채상환계수'] == np.inf, '부채상환계수'] = np.nan
df_final_1['부채상환계수'].fillna(df_final_1['부채상환계수'].max(), inplace=True)

## 2) 이자보상배율
## -inf 값은 0으로 대체
df_final_1.loc[df_final_1['이자보상배율'] == -np.inf, '이자보상배율'] = 0
## inf 값은 이자보상배율 중 최대값으로 대체
df_final_1.loc[df_final_1['이자보상배율'] == np.inf, '이자보상배율'] = np.nan
df_final_1['이자보상배율'].fillna(df_final_1['이자보상배율'].max(), inplace=True)

## 3) 재고자산회전율 0으로 대체
df_final_1.loc[df_final_1['재고자산회전율'] == np.inf, '재고자산회전율'] = 0

In [26]:
df_final_1.to_csv('./data/df_final.csv')

## 3-3) oversampling

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('./data/kosdaq_전처리.csv',index_col=0)
df['거래소코드'] = df['거래소코드'].astype('str').str.zfill(6)
df['회계년도'] = pd.to_datetime(df['회계년도'],format='%Y')
df['회계년도'] = df['회계년도'].dt.year

In [43]:
df_over = df.copy()

In [44]:
# 1) 2021년 회계년도 삭제
df_over.loc[df_over['회계년도']==2021,'회계년도']=np.NaN
df_over.dropna(subset=['회계년도'],axis=0,inplace=True)
df_over['회계년도'] = pd.to_datetime(df_over['회계년도'],format='%Y')
df_over['회계년도'] = df_over['회계년도'].dt.year
df_over.reset_index(drop=True,inplace=True)

In [65]:
# 2)무한대값 재확인
np.isinf(abs(df_over.iloc[:,5:37])).sum()

y            0
자기자본비율       0
부채비율         0
유동비율         0
당좌비율         0
차입금의존도       0
매출액규모        0
총자산규모        0
부채상환계수       0
총자본순이익률      0
자본금세전순이익률    0
자기자본순이익률     0
매출액세전순이익률    0
매출액영업이익률     0
금융비용대비매출액    0
매출원가율        0
이자보상배율       0
총자본회전율       0
자기자본회전율      0
비유동자산회전율     0
유형고정자산회전율    2
매출채권회전율      0
재고자산회전율      0
운전자산회전율      0
총자산증가율       0
유형자산증가율      3
매출액증가율       3
순이익증가율       0
자기자본증가율      0
총자본투자효율      0
부가가치율        0
노동소득분배율      0
dtype: int64

In [45]:
# 2) inf,-inf 값 대체
## 1) 부채상환계수
## -inf 값은 0으로 대체
df_over.loc[df_over['부채상환계수'] == -np.inf, '부채상환계수'] = 0.0
## inf 값은 부채상환계수 중 최대값으로 대체
df_over.loc[df_over['부채상환계수'] == np.inf, '부채상환계수'] = np.nan
df_over['부채상환계수'].fillna(df_over['부채상환계수'].max(), inplace=True)

## 2) 이자보상배율
## -inf 값은 0으로 대체
df_over.loc[df_over['이자보상배율'] == -np.inf, '이자보상배율'] = 0.0
## inf 값은 이자보상배율 중 최대값으로 대체
df_over.loc[df_over['이자보상배율'] == np.inf, '이자보상배율'] = np.nan
df_over['이자보상배율'].fillna(df_over['이자보상배율'].max(), inplace=True)

## 3) 재고자산회전율 0으로 대체
df_over.loc[df_over['재고자산회전율'] == np.inf, '재고자산회전율'] = 0

## 4) 유형고정자산회전율
## inf 값은 유형고정자산 회전이 안된다는 뜻이므로 0으로 두기
df_over.loc[df_over['유형고정자산회전율'] == np.inf, '유형고정자산회전율'] = 0.0
## 5) 유형자산증가율
## inf 값은 유형자산증가율 중 최대값으로 대체
df_over.loc[df_over['유형자산증가율'] == np.inf, '유형자산증가율'] = np.nan
df_over['유형자산증가율'].fillna(df_over['유형자산증가율'].max(), inplace=True)
## 6) 매출액 증가율
## 전기 매출액 =0 -> 연구개발기업, 특례상장기업 -> 매출액 증가율 = 0 으로 놓기
df_over.loc[df_over['매출액증가율'] == np.inf, '매출액증가율'] = 0.0

In [46]:
# 3) 감사의견코드 UQ -> 1 / 나머지 0
df_over['감사의견코드'] = np.where(df_over['감사의견코드'] == 'UQ', 1, 0)
df_over.head()

,회사명,거래소코드,회계년도,산업분류,감사의견코드,y,자기자본비율,부채비율,유동비율,당좌비율,...,운전자산회전율,총자산증가율,유형자산증가율,매출액증가율,순이익증가율,자기자본증가율,총자본투자효율,부가가치율,노동소득분배율,현금흐름
0,(주)CMG제약,58820,2011,제조업,1,0.0,75.627489,47.267492,230.996351,77.287792,...,146.669141,25.669219,9.441161,53.831859,831.580106,158.833603,12.634668,27.239028,56.204155,-3.379967
1,(주)CMG제약,58820,2012,제조업,1,0.0,41.797464,190.414329,424.286941,92.831089,...,47.927974,54.407046,-3.004507,0.747696,-687.677616,-21.701042,-5.113080,-15.501043,-0.000000,-11.284429
2,(주)CMG제약,58820,2013,제조업,1,0.0,95.597728,16.321186,633.696024,88.098714,...,44.257043,25.226274,11.193917,22.780406,720.967790,212.647290,6.852398,23.128577,58.087209,-7.429570
3,(주)CMG제약,58820,2014,제조업,1,0.0,85.914846,17.053378,422.691875,83.270423,...,77.520482,1.138907,76.940488,11.870458,-50.813199,0.506265,5.258565,17.743146,76.622734,0.189064
4,(주)CMG제약,58820,2015,제조업,1,0.0,86.334675,16.442479,358.939268,81.390646,...,100.565231,1.066129,-6.083943,15.520101,-369.911312,1.596356,4.993929,14.747191,85.089072,2.766692


In [47]:
# 4) '감사의견코드','산업분류','y' 카테고리로 만들기
df_over[['감사의견코드','산업분류','y']] = df_over[['감사의견코드','산업분류','y']].astype('category')

In [68]:
# 5) 연속형변수+범주형변수 -> SMOTENC로 oversampling
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC

features = df_over.drop(['회사명','거래소코드','회계년도','y'], axis=1)
target = df_over[['y']]

X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=121, stratify=target, test_size=0.2)
X_train_over, y_train_over = SMOTENC(sampling_strategy = 'auto', categorical_features=(X_train.dtypes == "category").values,random_state=121).fit_resample(X_train, y_train)

In [69]:
print('SMOTE 적용 전 학습용 피처/테스트용 피처 데이터 세트: ', X_train.shape, X_test.shape)
print('SMOTE 적용 후 학습용 피처/테스트용 피처 데이터 세트: ', X_train_over.shape, X_test.shape)

SMOTE 적용 전 학습용 피처/테스트용 피처 데이터 세트:  (8527, 34) (2132, 34)
SMOTE 적용 후 학습용 피처/테스트용 피처 데이터 세트:  (16304, 34) (2132, 34)


In [79]:
print('raw data 관리종목 / 비관리종목 데이터 세트: ', (df_raw[df_raw['y']==1]['회사명'].count(), df_raw[df_raw['y']==0]['회사명'].count()))
print('Undersampling 후 관리종목 / 비관리종목 데이터 세트: ', (df_final_1[df_final_1['y']==1]['회사명'].count(), df_final_1[df_final_1['y']==0]['회사명'].count()))

raw data 관리종목 / 비관리종목 데이터 세트:  (469, 10190)
Undersampling 후 관리종목 / 비관리종목 데이터 세트:  (469, 469)


In [74]:
X_train_over.to_csv('./data/X_train_over.csv')
y_train_over.to_csv('./data/y_train_over.csv')
df_over.to_csv('./data/df_over.csv')

# 4. robust 검정을 위해 df_raw 만들기

In [2]:
df_raw = df.copy()

In [3]:
# 1) 2021년 회계년도 삭제
df_raw.loc[df_raw['회계년도']==2021,'회계년도']=np.NaN
df_raw.dropna(subset=['회계년도'],axis=0,inplace=True)
df_raw['회계년도'] = pd.to_datetime(df_raw['회계년도'],format='%Y')
df_raw['회계년도'] = df_raw['회계년도'].dt.year
df_raw.reset_index(drop=True,inplace=True)

In [4]:
# 2) inf,-inf 값 대체
## 1) 부채상환계수
## -inf 값은 0으로 대체
df_raw.loc[df_raw['부채상환계수'] == -np.inf, '부채상환계수'] = 0.0
# inf 값은 부채상환계수 중 최대값으로 대체
df_raw.loc[df_raw['부채상환계수'] == np.inf, '부채상환계수'] = np.nan
df_raw['부채상환계수'].fillna(df_raw['부채상환계수'].max(), inplace=True)

## 2) 이자보상배율
## -inf 값은 0으로 대체
df_raw.loc[df_raw['이자보상배율'] == -np.inf, '이자보상배율'] = 0.0
## inf 값은 이자보상배율 중 최대값으로 대체
df_raw.loc[df_raw['이자보상배율'] == np.inf, '이자보상배율'] = np.nan
df_raw['이자보상배율'].fillna(df_raw['이자보상배율'].max(), inplace=True)

## 3) 재고자산회전율 0으로 대체
df_raw.loc[df_raw['재고자산회전율'] == np.inf, '재고자산회전율'] = 0

## 4) 유형고정자산회전율
## inf 값은 유형고정자산 회전이 안된다는 뜻이므로 0으로 두기
df_raw.loc[df_raw['유형고정자산회전율'] == np.inf, '유형고정자산회전율'] = 0.0
## 5) 유형자산증가율
## inf 값은 유형자산증가율 중 최대값으로 대체
df_raw.loc[df_raw['유형자산증가율'] == np.inf, '유형자산증가율'] = np.nan
df_raw['유형자산증가율'].fillna(df_raw['유형자산증가율'].max(), inplace=True)
## 6) 매출액 증가율
## 전기 매출액 =0 -> 연구개발기업, 특례상장기업 -> 매출액 증가율 = 0 으로 놓기
df_raw.loc[df_raw['매출액증가율'] == np.inf, '매출액증가율'] = 0.0

In [5]:
# 3) 감사의견코드 UQ -> 1 / 나머지 0
df_raw['감사의견코드'] = np.where(df_raw['감사의견코드'] == 'UQ', 1, 0)
df_raw.head()

,회사명,거래소코드,회계년도,산업분류,감사의견코드,y,자기자본비율,부채비율,유동비율,당좌비율,...,운전자산회전율,총자산증가율,유형자산증가율,매출액증가율,순이익증가율,자기자본증가율,총자본투자효율,부가가치율,노동소득분배율,현금흐름
0,(주)CMG제약,058820,2011,제조업,1,0.0,75.627489,47.267492,230.996351,77.287792,...,146.669141,25.669219,9.441161,53.831859,831.580106,158.833603,12.634668,27.239028,56.204155,-3.379967
1,(주)CMG제약,058820,2012,제조업,1,0.0,41.797464,190.414329,424.286941,92.831089,...,47.927974,54.407046,-3.004507,0.747696,-687.677616,-21.701042,-5.113080,-15.501043,-0.000000,-11.284429
2,(주)CMG제약,058820,2013,제조업,1,0.0,95.597728,16.321186,633.696024,88.098714,...,44.257043,25.226274,11.193917,22.780406,720.967790,212.647290,6.852398,23.128577,58.087209,-7.429570
3,(주)CMG제약,058820,2014,제조업,1,0.0,85.914846,17.053378,422.691875,83.270423,...,77.520482,1.138907,76.940488,11.870458,-50.813199,0.506265,5.258565,17.743146,76.622734,0.189064
4,(주)CMG제약,058820,2015,제조업,1,0.0,86.334675,16.442479,358.939268,81.390646,...,100.565231,1.066129,-6.083943,15.520101,-369.911312,1.596356,4.993929,14.747191,85.089072,2.766692


In [6]:
# 4) 산업분류 -> n-1개의 더미변수 만들기
df_raw = pd.concat([df_raw,pd.get_dummies(df_raw['산업분류'],drop_first=True)],axis=1)
df_raw.drop('산업분류',axis=1,inplace=True)
df_raw.head()

,회사명,거래소코드,회계년도,감사의견코드,y,자기자본비율,부채비율,유동비율,당좌비율,차입금의존도,...,도매 및 소매업,"사업시설 관리, 사업 지원 및 임대 서비스업","수도, 하수 및 폐기물 처리, 원료 재생업",숙박 및 음식점업,"예술, 스포츠 및 여가관련 서비스업",운수 및 창고업,"전기, 가스, 증기 및 공기조절 공급업","전문, 과학 및 기술 서비스업",정보통신업,제조업
0,(주)CMG제약,058820,2011,1,0.0,75.627489,47.267492,230.996351,77.287792,7.556527,...,0,0,0,0,0,0,0,0,0,1
1,(주)CMG제약,058820,2012,1,0.0,41.797464,190.414329,424.286941,92.831089,0.000000,...,0,0,0,0,0,0,0,0,0,1
2,(주)CMG제약,058820,2013,1,0.0,95.597728,16.321186,633.696024,88.098714,0.013432,...,0,0,0,0,0,0,0,0,0,1
3,(주)CMG제약,058820,2014,1,0.0,85.914846,17.053378,422.691875,83.270423,0.007751,...,0,0,0,0,0,0,0,0,0,1
4,(주)CMG제약,058820,2015,1,0.0,86.334675,16.442479,358.939268,81.390646,0.000000,...,0,0,0,0,0,0,0,0,0,1


In [7]:
df_raw_scale = df_raw.iloc[:,5:37]

In [8]:
# 5) standard scale로 변화를 확인해보고자 원본 독립변수들을 스케일링 진행
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()
x_scaled = scale.fit_transform(df_raw_scale)
x_scaled_df = pd.DataFrame(x_scaled, columns = df_raw_scale.columns)
x_scaled_df.head()

,자기자본비율,부채비율,유동비율,당좌비율,차입금의존도,매출액규모,총자산규모,부채상환계수,총자본순이익률,자본금세전순이익률,...,운전자산회전율,총자산증가율,유형자산증가율,매출액증가율,순이익증가율,자기자본증가율,총자본투자효율,부가가치율,노동소득분배율,현금흐름
0,0.600115,-0.088611,-0.152177,-0.068500,-0.529329,-1.190786,-1.402535,-0.265017,0.129743,0.058302,...,0.006998,-0.008603,-0.022632,-0.004544,0.095492,2.013950,0.652684,0.020788,0.007196,-0.051983
1,-0.903875,0.128345,0.191952,0.834615,-1.037964,-1.184809,-0.944775,-0.265039,-1.039149,-1.099043,...,0.001341,-0.006179,-0.023074,-0.023999,-0.073345,-0.494865,-1.350711,-0.005536,-0.028856,-0.143881
2,1.487938,-0.135514,0.564778,0.559649,-1.037060,-1.020142,-0.707738,-0.265037,0.013648,-0.050532,...,0.001131,-0.008640,-0.022569,-0.015924,0.083200,2.761776,-0.000028,0.018256,0.008404,-0.099064
3,1.057463,-0.134404,0.189112,0.279110,-1.037442,-0.930140,-0.695805,-0.265034,-0.048508,-0.118188,...,0.003037,-0.010672,-0.020231,-0.019923,-0.002570,-0.186259,-0.179943,0.014939,0.020293,-0.010489
4,1.076128,-0.135330,0.075609,0.169889,-1.037964,-0.814379,-0.684630,-0.265035,-0.227591,-0.295503,...,0.004357,-0.010678,-0.023184,-0.018585,-0.038032,-0.171111,-0.209815,0.013094,0.025724,0.019479


In [9]:
# 6) boxplot에서 보이는 이상치 처리 -> 5% winsorizing 활용
from scipy.stats.mstats import winsorize
for i in range(len(x_scaled_df.columns)):
  x_scaled_df.iloc[:,i]= winsorize(x_scaled_df.iloc[:,i], limits=[0.025, 0.025])

In [10]:
df_raw_final = pd.concat([df_raw[x_scaled_df.columns],df_raw[['회사명', '거래소코드', '회계년도', '감사의견코드', 'y','광업', '교육 서비스업', '금융 및 보험업',
       '농업, 임업 및 어업', '도매 및 소매업', '사업시설 관리, 사업 지원 및 임대 서비스업',
       '수도, 하수 및 폐기물 처리, 원료 재생업', '숙박 및 음식점업', '예술, 스포츠 및 여가관련 서비스업',
       '운수 및 창고업', '전기, 가스, 증기 및 공기조절 공급업', '전문, 과학 및 기술 서비스업', '정보통신업',
       '제조업']]],axis=1)

In [14]:
df_raw_final

,자기자본비율,부채비율,유동비율,당좌비율,차입금의존도,매출액규모,총자산규모,부채상환계수,총자본순이익률,자본금세전순이익률,...,도매 및 소매업,"사업시설 관리, 사업 지원 및 임대 서비스업","수도, 하수 및 폐기물 처리, 원료 재생업",숙박 및 음식점업,"예술, 스포츠 및 여가관련 서비스업",운수 및 창고업,"전기, 가스, 증기 및 공기조절 공급업","전문, 과학 및 기술 서비스업",정보통신업,제조업
0,75.627489,47.267492,230.996351,77.287792,7.556527,16.323210,17.199146,6.893390e+03,2.162576,2.162576,...,0,0,0,0,0,0,0,0,0,1
1,41.797464,190.414329,424.286941,92.831089,0.000000,16.330659,17.633568,-3.814426e+02,-8.970669,-8.970669,...,0,0,0,0,0,0,0,0,0,1
2,95.597728,16.321186,633.696024,88.098714,0.013432,16.535886,17.858520,2.230817e+02,1.056814,1.115638,...,0,0,0,0,0,0,0,0,0,1
3,85.914846,17.053378,422.691875,83.270423,0.007751,16.648058,17.869845,1.440493e+03,0.464809,0.464809,...,0,0,0,0,0,0,0,0,0,1
4,86.334675,16.442479,358.939268,81.390646,0.000000,16.792332,17.880450,9.365915e+02,-1.240894,-1.240894,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10654,88.159174,14.979385,512.170957,85.348949,7.728559,18.467144,18.181074,1.366841e+02,1.760619,2.210060,...,1,0,0,0,0,0,0,0,0,0
10655,87.218955,14.947469,170.530914,83.775334,1.267496,18.558622,18.186194,1.780308e+01,1.353920,1.709123,...,1,0,0,0,0,0,0,0,0,0
10656,88.579677,11.376688,190.727293,75.544440,3.843712,18.641085,18.159334,3.486386e+01,1.270174,1.592513,...,1,0,0,0,0,0,0,0,0,0
10657,95.366942,8.628975,448.803427,89.984485,0.000000,18.515681,18.231288,1.295423e+09,9.359543,12.360547,...,1,0,0,0,0,0,0,0,0,0


In [62]:
# df_raw_final.to_csv('./data/df_raw_scaled_df.csv')

In [46]:
# df_raw.to_csv('./data/df_raw.csv')